In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_json('ML-ESG-2_French_Train.json')
df.head()

,URL,news_title,news_content,impact_type
0,https://www.novethic.fr/actualite/energie/mobi...,Chaos dans les transports publics : le pari ri...,"Journées de 13 heures, conflits à gérer, salai...",Risk
1,https://www.novethic.fr/actualite/energie/mobi...,Chaos dans les transports publics : le pari ri...,L'ouverture à la concurrence des transports en...,Risk
2,https://www.novethic.fr/actualite/energie/mobi...,Chaos dans les transports publics : le pari ri...,"""Les opérateurs en compétition contractent les...",Opportunity
3,https://www.novethic.fr/actualite/energie/mobi...,Chaos dans les transports publics : le pari ri...,"Toutefois, les salariés s'inquiètent. ""Les opé...",Risk
4,https://www.novethic.fr/actualite/energie/mobi...,Chaos dans les transports publics : le pari ri...,"""La profession s'est tendue""\nLes conditions d...",Risk


In [3]:
from newspaper import Article

def get_article_text(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

In [4]:
# texts = df['URL'].apply(get_article_text)

In [5]:
from nltk.corpus import stopwords

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

scores = []
X = df['news_content']
y = df['impact_type'].map({'Opportunity': 0, 'Risk': 1}).values
for c in np.arange(2, 7):
    mdl = make_pipeline(
        TfidfVectorizer(max_features=1000, stop_words=stopwords.words('french')),
        LogisticRegression(C=10**c)
    )

    scores.append(
        cross_validate(mdl, X, y, cv=5, n_jobs=5, scoring=['precision', 'recall', 'f1'])
    )

/Users/aliakbars/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/aliakbars/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [7]:
scores = pd.DataFrame(scores)
scores['precision'] = scores['test_precision'].apply(np.mean)
scores['recall'] = scores['test_recall'].apply(np.mean)
scores['f1'] = scores['test_f1'].apply(np.mean)

In [8]:
scores[['precision', 'recall', 'f1']].assign(c=np.arange(2, 7))

,precision,recall,f1,c
0,0.512271,0.508333,0.508172,2
1,0.499388,0.502778,0.499851,3
2,0.492439,0.491667,0.490820,4
3,0.508203,0.502778,0.503635,5
4,0.512167,0.500000,0.504627,6


In [53]:
from sklearn.ensemble import RandomForestClassifier

mdl = make_pipeline(
    TfidfVectorizer(max_features=1000),
    RandomForestClassifier(n_estimators=10, random_state=42)
)

cross_validate(mdl, X, y, cv=5, n_jobs=5, scoring=['precision', 'recall', 'f1'])

{'fit_time': array([0.05986905, 0.05568218, 0.05850768, 0.06352997, 0.0595901 ]),
 'score_time': array([0.0073688 , 0.00769567, 0.0079453 , 0.00793695, 0.00833774]),
 'test_precision': array([0.53030303, 0.52941176, 0.48648649, 0.53333333, 0.57692308]),
 'test_recall': array([0.48611111, 0.375     , 0.25      , 0.33333333, 0.20833333]),
 'test_f1': array([0.50724638, 0.43902439, 0.33027523, 0.41025641, 0.30612245])}

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('dangvantuan/sentence-camembert-base')

In [8]:
content_embeddings = model.encode(df['news_content'].values)

## Camembert-Base

In [9]:
results = cross_validate(LogisticRegression(C=10), content_embeddings, y, cv=5, n_jobs=5, scoring=['precision', 'recall', 'f1'])
results

{'fit_time': array([0.08080626, 0.07078195, 0.06163931, 0.05996609, 0.05925393]),
 'score_time': array([0.00407505, 0.01048017, 0.00217295, 0.00555801, 0.00231409]),
 'test_precision': array([0.5257732 , 0.6       , 0.6       , 0.59615385, 0.62857143]),
 'test_recall': array([0.70833333, 0.54166667, 0.54166667, 0.43055556, 0.61111111]),
 'test_f1': array([0.6035503 , 0.56934307, 0.56934307, 0.5       , 0.61971831])}

In [11]:
print(f"{results['test_f1'].mean():.2%} ({results['test_f1'].std():.2%})")

57.24% (4.12%)


## Sentence-T5

In [13]:
t5 = SentenceTransformer('sentence-transformers/sentence-t5-base')
t5_embeddings = t5.encode(df['news_content'].values)

In [22]:
results = cross_validate(LogisticRegression(C=10), t5_embeddings, y, cv=5, n_jobs=5, scoring=['precision', 'recall', 'f1'])
results

{'fit_time': array([0.03769803, 0.04474688, 0.03408504, 0.04611206, 0.04028487]),
 'score_time': array([0.00144386, 0.00201821, 0.00148106, 0.00173807, 0.00122428]),
 'test_precision': array([0.59550562, 0.62711864, 0.53061224, 0.62222222, 0.68      ]),
 'test_recall': array([0.73611111, 0.51388889, 0.36111111, 0.38888889, 0.47222222]),
 'test_f1': array([0.65838509, 0.5648855 , 0.42975207, 0.47863248, 0.55737705])}

In [23]:
print(f"{results['test_f1'].mean():.2%} ({results['test_f1'].std():.2%})")

53.78% (7.85%)
